In [1]:
#Import and install tools

!pip install pandasql
import os
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt
from sqlite3 import connect
from scipy.stats.mstats import winsorize, ks_2samp
import seaborn as sns
import pandasql as ps

conn = connect(':memory:', timeout = 10)
filePath = "/Users/aaron/AaronTuFIMA/FamaFrenchFiveFactors"

In [2]:
# import CRSP data
file = filePath + "/CRSPMonthly1990Through2022.csv"
crsp1990To2022 = pd.read_csv(file)
print(crsp1990To2022)
print(crsp1990To2022.columns)

C:\Users\aaron\AppData\Local\Temp\ipykernel_27140\2350463996.py:3: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990To2022 = pd.read_csv(file)


         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920  39040610   GFGC   
3         10001  19900430       NaN   11.0     3.0    4920  39040610   GFGC   
4         10001  19900531       NaN   11.0     3.0    4920  39040610   GFGC   
...         ...       ...       ...    ...     ...     ...       ...    ...   
2975236   93436  20211130       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975237   93436  20211231       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975238   93436  20220131       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975239   93436  20220228       NaN   11.0     3.0  9999.0  88160R10   TSLA   
2975240   93436  20220331       NaN   11.0     3.0  9999.0  88160R10   TSLA   

                     COMNAM SHRCLS  ... CFACSHR    

In [3]:
#Clean CRSP Dataframe

#Parse relevant variables
crsp1990To2022.to_sql("crsp1990To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT PERMNO as permno, date, RET as ret, SHROUT as shrout, ALTPRC as altprc, EXCHCD as exchcd,
        SHRCD as shrcd, SICCD as siccd, DLRET as dlret, DLSTCD as dlstcd
        FROM crsp1990To2022
        '''
crsp1990To2022 = pd.read_sql(query, conn)

In [4]:
#Convert each column to correct variable types
crsp1990To2022['permno'] = pd.to_numeric(crsp1990To2022['permno'], errors = 'coerce') #security identifier
crsp1990To2022['date'] = pd.to_datetime(crsp1990To2022['date'], format = '%Y%m%d') #month identifier
crsp1990To2022['ret'] = pd.to_numeric(crsp1990To2022['ret'], errors = 'coerce') #return
crsp1990To2022['shrout'] = pd.to_numeric(crsp1990To2022['shrout'], errors = 'coerce') #share outstanding (thousands)
crsp1990To2022['altprc'] = pd.to_numeric(crsp1990To2022['altprc'], errors = 'coerce') #last traded price in a month
crsp1990To2022['exchcd'] = pd.to_numeric(crsp1990To2022['exchcd'], errors = 'coerce') #exchange code
crsp1990To2022['shrcd'] = pd.to_numeric(crsp1990To2022['shrcd'], errors = 'coerce') #share code
crsp1990To2022['siccd'] = pd.to_numeric(crsp1990To2022['siccd'], errors = 'coerce') #industry code
crsp1990To2022['dlret'] = pd.to_numeric(crsp1990To2022['dlret'], errors = 'coerce') #delisting return
crsp1990To2022['dlstcd'] = pd.to_numeric(crsp1990To2022['dlstcd'], errors = 'coerce') #delisting code

In [5]:
#Convert returns to percent
crsp1990To2022['ret'] *= 100
crsp1990To2022['dlret'] *= 100

#only keep data after 2018
crsp2018To2022 = crsp1990To2022[crsp1990To2022['date'] >= "2018-01-01"]


In [6]:
#Only keep US-based common stocks
crsp2018To2022 = crsp2018To2022[(crsp2018To2022['shrcd'] == 10) | (crsp2018To2022['shrcd'] == 11)]

#calculate market cap (millions)
crsp2018To2022['market_cap'] = abs(crsp2018To2022['shrout'] * crsp2018To2022['altprc']) / 1000
crsp2018To2022['market_cap'].replace(0, np.NaN)

2358       2.585782e+03
2359       2.509016e+03
2360       2.553262e+03
2361       2.569842e+03
2362       2.648577e+03
               ...     
2947235    1.149642e+06
2947236    1.092218e+06
2947237    9.681319e+05
2947238    8.996190e+05
2947239    1.113736e+06
Name: market_cap, Length: 194624, dtype: float64

In [7]:
# Add exchange column, define exchange labels and keep only relevant labels
crsp2018To2022['exchange'] = ""
for i in crsp2018To2022.index:
    if crsp2018To2022['exchcd'][i] in [1, 31]:
        crsp2018To2022.loc[i, 'exchange'] = "NYSE"
    elif crsp2018To2022['exchcd'][i] in [2, 32]:
        crsp2018To2022.loc[i, 'exchange'] = "AMEX"
    elif crsp2018To2022['exchcd'][i] in [3, 33]:
        crsp2018To2022.loc[i, 'exchange'] = "NASDAQ"
    else:
        crsp2018To2022.loc[i, 'exchange'] = "Other"

crsp2018To2022 = crsp2018To2022[crsp2018To2022['exchange'] != "Other"]

In [8]:
#add adjusted Return Column, adjust delisting returns
crsp2018To2022['ret_adj'] = ""
for i in crsp2018To2022.index:
    if pd.isnull(crsp2018To2022['dlstcd'][i]):
        crsp2018To2022.loc[i, 'ret_adj'] = crsp2018To2022.loc[i, 'ret']
    elif (pd.notnull(crsp2018To2022['dlstcd'][i])) and (pd.notnull(crsp2018To2022['dlret'][i])):
        crsp2018To2022.loc[i, 'ret_adj'] = crsp2018To2022.loc[i, 'dlret']
    elif (551 <= crsp2018To2022.loc[i, 'dlstcd'] <= 574) or (crsp2018To2022.loc[i, 'dlstcd'] in [500, 520, 580, 584]):
        crsp2018To2022.loc[i, 'ret_adj'] = -30
    else:
        crsp2018To2022.loc[i, 'ret_adj'] = -100

In [9]:
#Reindex and drop unneeded columns
crsp2018To2022 = crsp2018To2022.reset_index()
crsp2018To2022.drop(['shrcd', 'dlret', 'dlstcd', 'index'], inplace = True, axis = 1)

print(crsp2018To2022)
print(crsp2018To2022.columns)

        permno       date      ret     shrout      altprc  exchcd   siccd  \
0        10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
1        10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2        10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
3        10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
4        10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...        ...        ...      ...        ...         ...     ...     ...   
193370   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
193371   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
193372   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
193373   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
193374   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

          market_cap exchange  ret_adj  
0       2.585782e+03   NASDAQ  -8.

In [10]:
#Import CRSP/Compustat Merged (CCM) Data

file = filePath + "/ccmfund1990to2022.csv"
ccmFund1990To2022 = pd.read_csv(file)
print(ccmFund1990To2022)
print(ccmFund1990To2022.columns)

C:\Users\aaron\AppData\Local\Temp\ipykernel_27140\2341482942.py:4: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccmFund1990To2022 = pd.read_csv(file)


         GVKEY LINKPRIM LIID LINKTYPE  LPERMNO  LPERMCO    LINKDT LINKENDDT  \
0         1004        P    1       LU    54594    20000  19720424         E   
1         1004        P    1       LU    54594    20000  19720424         E   
2         1004        P    1       LU    54594    20000  19720424         E   
3         1004        P    1       LU    54594    20000  19720424         E   
4         1004        P    1       LU    54594    20000  19720424         E   
...        ...      ...  ...      ...      ...      ...       ...       ...   
209121  345980        P    1       LC    20333    57666  20201216         E   
209122  345980        P    1       LC    20333    57666  20201216         E   
209123  347007        P    1       LC    15533    55364  20210310         E   
209124  347085        P    1       LC    21069    58208  20210401         E   
209125  349530        P    1       LC    17324    56274  20210709         E   

        datadate   fyear  ... priusa     sic spcind

In [11]:
#Clean CRSP/Compustat merged Dataframe

#parse relevant Variables
ccmFund1990To2022.to_sql("ccmFund1990To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY, LPERMNO, datadate, LINKTYPE, LINKENDDT, seq, ceq, at, lt, txditc,
                        txdb, itcb, pstkrv, pstkl, pstk, indfmt, datafmt, revt, cogs, xint, xsga
        FROM ccmFund1990To2022
        '''
ccmFund1990To2022 = pd.read_sql(query, conn)

In [12]:
#Convert Variables
ccmFund1990To2022['GVKEY'] = pd.to_numeric(ccmFund1990To2022['GVKEY'], errors = 'coerce') #firm identifier
ccmFund1990To2022['LPERMNO'] = pd.to_numeric(ccmFund1990To2022['LPERMNO'], errors = 'coerce') #stock identifier
ccmFund1990To2022['datadate'] = pd.to_datetime(ccmFund1990To2022['datadate'], format = '%Y%m%d') #date of report
ccmFund1990To2022['LINKTYPE'] = ccmFund1990To2022['LINKTYPE'].apply(str) #link type
ccmFund1990To2022['LINKENDDT'] = pd.to_datetime(ccmFund1990To2022['LINKENDDT'], format = '%Y%m%d', errors = 'coerce') #link's date of expiration
ccmFund1990To2022['seq'] = pd.to_numeric(ccmFund1990To2022['seq'], errors = 'coerce') #stockholder's equity
ccmFund1990To2022['ceq'] = pd.to_numeric(ccmFund1990To2022['ceq'], errors = 'coerce') #total equity
ccmFund1990To2022['at'] = pd.to_numeric(ccmFund1990To2022['at'], errors = 'coerce') #total assets
ccmFund1990To2022['lt'] = pd.to_numeric(ccmFund1990To2022['lt'], errors = 'coerce') #total liabilities
ccmFund1990To2022['txditc'] = pd.to_numeric(ccmFund1990To2022['txditc'], errors = 'coerce') #deferred taxes and investment tax credit
ccmFund1990To2022['txdb'] = pd.to_numeric(ccmFund1990To2022['txdb'], errors = 'coerce') #deferred taxes
ccmFund1990To2022['itcb'] = pd.to_numeric(ccmFund1990To2022['itcb'], errors = 'coerce') #investment tax credit
ccmFund1990To2022['pstkrv'] = pd.to_numeric(ccmFund1990To2022['pstkrv'], errors = 'coerce') #preferred stock redemption value
ccmFund1990To2022['pstkl'] = pd.to_numeric(ccmFund1990To2022['pstkl'], errors = 'coerce') #preferred stock liquidating value
ccmFund1990To2022['pstk'] = pd.to_numeric(ccmFund1990To2022['pstk'], errors = 'coerce') #preffered stock par value
ccmFund1990To2022['indfmt'] = ccmFund1990To2022['indfmt'].apply(str) #industry format
ccmFund1990To2022['datafmt'] = ccmFund1990To2022['datafmt'].apply(str) #data format
ccmFund1990To2022['revt'] = pd.to_numeric(ccmFund1990To2022['revt'], errors = 'coerce') #total revenue
ccmFund1990To2022['cogs'] = pd.to_numeric(ccmFund1990To2022['cogs'], errors = 'coerce') #cost of goods sold
ccmFund1990To2022['xint'] = pd.to_numeric(ccmFund1990To2022['xint'], errors = 'coerce') #interest expense
ccmFund1990To2022['xsga'] = pd.to_numeric(ccmFund1990To2022['xsga'], errors = 'coerce') #selling, general, and administrative expenses

#Filter Columns

#keep only data after 2017 (1 year earlier needed for investment)
ccmFund2018To2022 = ccmFund1990To2022[ccmFund1990To2022['datadate'] >= "2017-01-01 00:00:00"]

#Only keep correct formats
ccmFund2018To2022 = ccmFund2018To2022[ccmFund2018To2022['indfmt'] == "INDL"]
ccmFund2018To2022 = ccmFund2018To2022[ccmFund2018To2022['datafmt'] == "STD"]

#Only keep valid links
ccmFund2018To2022 = ccmFund2018To2022[(ccmFund2018To2022['LINKTYPE'] == "LU") | (ccmFund2018To2022['LINKTYPE'] == "LC")]

#Only keep links active at datadate
ccmFund2018To2022 = ccmFund2018To2022[(ccmFund2018To2022['datadate'] <= ccmFund2018To2022['LINKENDDT']) | pd.isnull(ccmFund2018To2022['LINKENDDT'])]

In [13]:
#Calculate book value and operating profitability
ccmFund2018To2022.to_sql("ccmFund2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT GVKEY as gvkey, LPERMNO as permno, datadate, at,
                        COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0) as book_value,
                        (revt - cogs - xint - xsga) / (COALESCE(seq, ceq + pstk, at - lt) + COALESCE(txditc, txdb + itcb, 0) - COALESCE(pstkrv, pstkl, pstk, 0)) as operating_profitability
        FROM ccmFund2018To2022
        '''
ccmFund2018To2022 = pd.read_sql(query, conn)
ccmFund2018To2022['book_value'] = pd.to_numeric(ccmFund2018To2022['book_value'], errors = 'coerce')
ccmFund2018To2022['operating_profitability'] = pd.to_numeric(ccmFund2018To2022['operating_profitability'], errors = 'coerce')

In [14]:
#Only keep the last observation of each year
ccmFund2018To2022['datadate'] = pd.to_datetime(ccmFund2018To2022['datadate'])
ccmFund2018To2022['year'] = ccmFund2018To2022['datadate'].dt.year
ccmFund2018To2022 = ccmFund2018To2022.sort_values(by = ['permno', 'datadate'], ascending = True)
ccmFund2018To2022 = ccmFund2018To2022.reset_index()
toDrop = []
for i in range(len(ccmFund2018To2022) - 1):
    if(ccmFund2018To2022.loc[i, 'permno'] == ccmFund2018To2022.loc[i + 1, 'permno']) and (ccmFund2018To2022.loc[i, 'year'] == ccmFund2018To2022.loc[i + 1, 'year']):
        toDrop.append(i)
ccmFund2018To2022.drop(toDrop, inplace = True, axis = 0)
ccmFund2018To2022.drop('index', inplace = True, axis = 1)

In [15]:
#Drop missing/infinite values
ccmFund2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
ccmFund2018To2022 = ccmFund2018To2022.dropna()

#Reindex
ccmFund2018To2022 = ccmFund2018To2022.reset_index()
ccmFund2018To2022.drop('index', inplace = True, axis = 1)

In [16]:
#calculate investment (INV)
ccmFund2018To2022['investment'] = ""
for i in range(1, len(ccmFund2018To2022)):
    if ccmFund2018To2022.loc[i, 'permno'] == ccmFund2018To2022.loc[i-1, 'permno']:
        ccmFund2018To2022.loc[i, 'investment'] = ccmFund2018To2022.loc[i, 'at'] / ccmFund2018To2022.loc[i-1, 'at']

In [17]:
#Drop missing/infinite values
ccmFund2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
ccmFund2018To2022 = ccmFund2018To2022.dropna()

#Reindex
ccmFund2018To2022 = ccmFund2018To2022.reset_index()
ccmFund2018To2022.drop('index', inplace = True, axis = 1)

In [18]:
#Add reference date for matching (6/1 of next calendar year)
ccmFund2018To2022['year'] = pd.to_numeric(ccmFund2018To2022['year'], errors = 'coerce')
ccmFund2018To2022['reference_date'] = ccmFund2018To2022['year'] + 1
ccmFund2018To2022['reference_date'] = ccmFund2018To2022['reference_date'].apply(str)
ccmFund2018To2022['reference_date'] += "-06-01"
ccmFund2018To2022['reference_date'] = pd.to_datetime(ccmFund2018To2022['reference_date'])
ccmFund2018To2022.drop('year', inplace = True, axis = 1)

print(ccmFund2018To2022)
print(ccmFund2018To2022.columns)

        gvkey  permno   datadate         at  book_value  \
0       12096   10028 2017-12-31     13.315       7.756   
1       12096   10028 2018-12-31     13.018       8.413   
2       12096   10028 2019-12-31     27.003      11.194   
3       12096   10028 2020-12-31     40.579      17.578   
4       12096   10028 2021-12-31     59.267      27.627   
...       ...     ...        ...        ...         ...   
18082  184996   93436 2017-12-31  28655.372    4237.312   
18083  184996   93436 2018-12-31  29739.614    4923.243   
18084  184996   93436 2019-12-31  34309.000    6618.000   
18085  184996   93436 2020-12-31  52148.000   22376.000   
18086  184996   93436 2021-12-31  62131.000   30213.000   

       operating_profitability investment reference_date  
0                     0.271532                2018-06-01  
1                     0.240818   0.977694     2019-06-01  
2                     0.306593   2.074282     2020-06-01  
3                     0.392309   1.502759     2021-06-0

In [19]:
# Construct stocks sample

#Parse relevant variables from CRSP
crsp2018To2022.to_sql("crsp2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, date, exchange, ret_adj as ret, market_cap
        FROM crsp2018To2022
        '''
stocks2018To2022 = pd.read_sql(query, conn)

#Drop missing/infinite values
stocks2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018To2022 = stocks2018To2022.dropna()

In [20]:
#Add reference date for each stock (last 6/1)
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocks2018To2022['reference_date'] = ""
for i in stocks2018To2022.index:
    if stocks2018To2022.loc[i, 'date'].month < 6:
        stocks2018To2022.loc[i, 'reference_date'] = str(stocks2018To2022.loc[i, 'date'].year - 1) + "-06-01"
    else:
        stocks2018To2022.loc[i, 'reference_date'] = str(stocks2018To2022.loc[i, 'date'].year) + "-06-01"
stocks2018To2022['reference_date'] = pd.to_datetime(stocks2018To2022['reference_date'])

In [21]:
#Merge compustat data into stocks data
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
ccmFund2018To2022.to_sql("ccmFund2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, ccmFund2018To2022.gvkey, ccmFund2018To2022.datadate,
                        ccmFund2018To2022.book_value, ccmFund2018To2022.operating_profitability, ccmFund2018To2022.investment
        FROM stocks2018To2022
        LEFT JOIN ccmFund2018To2022
        ON stocks2018To2022.permno = ccmFund2018To2022.permno
        AND stocks2018To2022.reference_date = ccmFund2018To2022.reference_date
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

In [22]:
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])

# Filter the DataFrame for rows with month equal to 12
stocksMarketEquity = stocks2018To2022[stocks2018To2022['date'].dt.month == 12].copy()

# Create the 'reference_date' column by adding 1 year to the 'date' column
stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].apply(str)
stocksMarketEquity['reference_date'] += "-06-01"
stocksMarketEquity['reference_date'] = pd.to_datetime(stocksMarketEquity['reference_date'])


In [23]:
stocksMarketEquity.to_sql("stocksMarketEquity", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap as market_equity
        FROM stocksMarketEquity
        '''
stocksMarketEquity = pd.read_sql(query, conn)
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
stocksMarketEquity.to_sql("stocksMarketEquity", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, stocksMarketEquity.market_equity
        FROM stocks2018To2022
        LEFT JOIN stocksMarketEquity
        ON stocks2018To2022.permno = stocksMarketEquity.permno
        AND stocks2018To2022.reference_date = stocksMarketEquity.reference_date
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)
stocks2018To2022 = stocks2018To2022.dropna()
stocks2018To2022 = stocks2018To2022.reset_index()
stocks2018To2022.drop('index', inplace = True, axis = 1)

In [24]:
#Calculate BM ratio
stocks2018To2022['bm_ratio'] = stocks2018To2022['book_value']/stocks2018To2022['market_equity']

#Add market cap of June of year y (of reference date) (used for value-weighted returns)
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocksWeight = stocks2018To2022[stocks2018To2022['date'].dt.month == 6]
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
stocksWeight.to_sql("stocksWeight", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, stocksWeight.market_cap as market_cap_weight
        FROM stocks2018To2022
        LEFT JOIN stocksWeight
        ON stocks2018To2022.permno = stocksWeight.permno
        AND stocks2018To2022.reference_date = stocksWeight.reference_date
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

In [25]:
#filter data
stocks2018To2022.replace([np.inf, -np.inf], np.nan, inplace = True)
stocks2018To2022 = stocks2018To2022.dropna()
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
stocks2018To2022 = stocks2018To2022[stocks2018To2022['date'] >= '2018-01-01 00:00:00']

In [26]:
### Size Sort ###

#Get NYSE stocks' sizes in June of each year
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
sizeBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
sizeBreakpoints.to_sql("sizeBreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, market_cap
        FROM sizeBreakpoints
        '''
sizeBreakpoints = pd.read_sql(query, conn)
sizeBreakpoints['reference_date'] = pd.to_datetime(sizeBreakpoints['reference_date'])
g = sizeBreakpoints.groupby(['reference_date'])

In [27]:
# Create new dataframe to sort sizes
sizeSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)].copy()

# Initialize 'size_median' column
sizeSorts['size_median'] = None

# Calculate median size for each year
for i in sizeSorts.index:
    reference_date = sizeSorts.loc[i, 'reference_date']
    group_reference = g.get_group(reference_date)
    size_median = group_reference['market_cap'].quantile(0.5)
    sizeSorts.loc[i, 'size_median'] = size_median


In [28]:
sizeSorts['size_portfolio'] = ""
for i in sizeSorts.index:
    if sizeSorts.loc[i, 'market_cap'] > sizeSorts.loc[i, 'size_median']:
        sizeSorts.loc[i, 'size_portfolio'] = "B"
    else:
        sizeSorts.loc[i, 'size_portfolio'] = "S"

In [29]:
#Left join size portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
sizeSorts.to_sql("sizeSorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, sizeSorts.size_portfolio
        FROM stocks2018To2022
        LEFT JOIN sizeSorts
        ON stocks2018To2022.reference_date = sizeSorts.reference_date
        AND stocks2018To2022.permno = sizeSorts.permno
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

print(stocks2018To2022)
print(stocks2018To2022.columns)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79500   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79501   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79502   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79503   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79504   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [30]:
### Value sort ###
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
valueBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
valueBreakpoints.to_sql("valueBreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, bm_ratio
        FROM valueBreakpoints
        '''
valueBreakpoints = pd.read_sql(query, conn)
valueBreakpoints['reference_date'] = pd.to_datetime(valueBreakpoints['reference_date'])
g = valueBreakpoints.groupby(['reference_date'])

In [31]:
valueSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)].copy()

#Calculate 30th adn 70th quantile of book values
valueSorts['value_q30'] = ""
valueSorts['value_q70'] = ""
for i in valueSorts.index:
    valueSorts.loc[i, 'value_q30'] = g.get_group(g.get_group(valueSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['bm_ratio'].quantile(0.3)
    valueSorts.loc[i, 'value_q70'] = g.get_group(g.get_group(valueSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['bm_ratio'].quantile(0.7)

In [32]:
valueSorts['value_portfolio'] = ""
for i in valueSorts.index:
    if valueSorts.loc[i, 'bm_ratio'] > valueSorts.loc[i, 'value_q70']:
        valueSorts.loc[i, 'value_portfolio'] = "H"
    elif valueSorts.loc[i, 'bm_ratio'] > valueSorts.loc[i, 'value_q30']:
        valueSorts.loc[i, 'value_portfolio'] = "M"
    else:
        valueSorts.loc[i, 'value_portfolio'] = "L"

In [33]:
#left Join value portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
valueSorts.to_sql("valueSorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, valueSorts.value_portfolio
        FROM stocks2018To2022
        LEFT JOIN valueSorts
        ON stocks2018To2022.reference_date = valueSorts.reference_date
        AND stocks2018To2022.permno = valueSorts.permno
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

print(stocks2018To2022)
print(stocks2018To2022.columns)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79500   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79501   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79502   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79503   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79504   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [34]:
### Robustness Sort ###

#Get robustness (operating profitability/op) breakpoints of NYSE stocks
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
robustBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
robustBreakpoints.to_sql("robustBreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, operating_profitability
        FROM robustBreakpoints
        '''
robustBreakpoints = pd.read_sql(query, conn)
robustBreakpoints['reference_date'] = pd.to_datetime(robustBreakpoints['reference_date'])
g = robustBreakpoints.groupby(['reference_date'])

In [35]:
#Create new dataframe to sort values
robustSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)].copy()

#Calculate the 30th and 70th quantile of operating profitabilities
robustSorts['robust_q30'] = ""
robustSorts['robust_q70'] = ""
for i in robustSorts.index:
    robustSorts.loc[i, 'robust_q30'] = g.get_group(g.get_group(robustSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['operating_profitability'].quantile(0.3)
    robustSorts.loc[i, 'robust_q70'] = g.get_group(g.get_group(robustSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['operating_profitability'].quantile(0.7)

In [36]:
#Sort values into robust, neutral, and weak portfolios
robustSorts['robust_portfolio'] = ""
for i in robustSorts.index:
    if robustSorts.loc[i, 'operating_profitability'] > robustSorts.loc[i, 'robust_q70']:
        robustSorts.loc[i, 'robust_portfolio'] = "R"
    elif robustSorts.loc[i, 'operating_profitability'] > robustSorts.loc[i, 'robust_q30']:
        robustSorts.loc[i, 'robust_portfolio'] = "N"
    else:
        robustSorts.loc[i, 'robust_portfolio'] = "W"
        
#Left join robust portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
robustSorts.to_sql("robustSorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, robustSorts.robust_portfolio
        FROM stocks2018To2022
        LEFT JOIN robustSorts
        ON stocks2018To2022.reference_date = robustSorts.reference_date
        AND stocks2018To2022.permno = robustSorts.permno
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

print(stocks2018To2022)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79500   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79501   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79502   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79503   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79504   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [37]:
### Investment Sort ###

#Get investment (inv) breakpoints of NYSE stocks
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
investmentBreakpoints = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6) & (stocks2018To2022['exchange'] == "NYSE")]
investmentBreakpoints.to_sql("investmentBreakpoints", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT permno, reference_date, investment
        FROM investmentBreakpoints
        '''
investmentBreakpoints = pd.read_sql(query, conn)
investmentBreakpoints['investment'] = pd.to_numeric(investmentBreakpoints['investment'], errors = 'coerce')
investmentBreakpoints['reference_date'] = pd.to_datetime(investmentBreakpoints['reference_date'])
g = investmentBreakpoints.groupby(['reference_date'])

print(investmentBreakpoints)

      permno reference_date  investment
0      10104     2019-06-01    1.016838
1      10104     2020-06-01    0.791970
2      10104     2021-06-01    1.061899
3      10145     2019-06-01    0.972822
4      10145     2020-06-01    1.015682
...      ...            ...         ...
2804   93423     2020-06-01    1.145079
2805   93423     2021-06-01    0.961892
2806   93426     2019-06-01    1.064694
2807   93426     2020-06-01    1.134903
2808   93426     2021-06-01    1.084970

[2809 rows x 3 columns]


In [38]:
#Create new dataframe to sort values
investmentSorts = stocks2018To2022[(stocks2018To2022['date'].dt.month == 6)].copy()

#Calculate the 30th and 70th quantile of operating profitabilities
investmentSorts['investment_q30'] = ""
investmentSorts['investment_q70'] = ""
for i in investmentSorts.index:
    investmentSorts.loc[i, 'investment_q30'] = g.get_group(g.get_group(investmentSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['investment'].quantile(0.3)
    investmentSorts.loc[i, 'investment_q70'] = g.get_group(g.get_group(investmentSorts.loc[i, 'reference_date']).reset_index().loc[0, 'reference_date'])['investment'].quantile(0.7)

In [42]:
investmentSorts = pd.DataFrame(investmentSorts)
print(investmentSorts)


       permno       date exchange      ret     market_cap  \
0       10028 2019-06-28     AMEX  80.5555      35.001200   
12      10028 2020-06-30     AMEX  69.9164     164.236400   
24      10028 2021-06-30     AMEX  12.1495     129.240000   
34      10032 2019-06-28   NASDAQ  17.8478    1721.156190   
46      10032 2020-06-30   NASDAQ   9.8723    2059.364160   
...       ...        ...      ...      ...            ...   
79449   93434 2020-06-30   NASDAQ   7.0422      76.227240   
79461   93434 2021-06-30   NASDAQ   2.2472     133.762720   
79471   93436 2019-06-28   NASDAQ  20.6848   40025.710071   
79483   93436 2020-06-30   NASDAQ  29.3186  200844.671160   
79495   93436 2021-06-30   NASDAQ   8.7137  668826.848940   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
12     2020-06-01 00:00:00   12096.0  2019-12-31 00:00:00      11.194   
24     2021-06-01 00:00:00   12096.0  2020-12-31

In [48]:
#Sort values into conservative, neutral, and aggresive portfolios
investmentSorts['investment'] = pd.to_numeric(investmentSorts['investment'])
investmentSorts['investment_q70'] = pd.to_numeric(investmentSorts['investment_q70'])
investmentSorts['investment_portfolio'] = ""
for i in investmentSorts.index:
    if investmentSorts.loc[i, 'investment'] > investmentSorts.loc[i, 'investment_q70']:
        investmentSorts.loc[i, 'investment_portfolio'] = "A"
    elif investmentSorts.loc[i, 'investment'] > investmentSorts.loc[i, 'investment_q30']:
        investmentSorts.loc[i, 'investment_portfolio'] = "N"
    else:
        investmentSorts.loc[i, 'investment_portfolio'] = "C"
        
#Left join investment portfolio back into stocks dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
investmentSorts.to_sql("investmentSorts", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT stocks2018To2022.*, investmentSorts.investment_portfolio
        FROM stocks2018To2022
        LEFT JOIN investmentSorts
        ON stocks2018To2022.reference_date = investmentSorts.reference_date
        AND stocks2018To2022.permno = investmentSorts.permno
        '''
stocks2018To2022 = pd.read_sql(query, conn)
stocks2018To2022.drop('index', inplace = True, axis = 1)

print(stocks2018To2022)

       permno                 date exchange       ret    market_cap  \
0       10028  2019-06-28 00:00:00     AMEX   80.5555  3.500120e+01   
1       10028  2019-07-31 00:00:00     AMEX  -37.8461  2.175459e+01   
2       10028  2019-08-30 00:00:00     AMEX   55.9406  3.392424e+01   
3       10028  2019-09-30 00:00:00     AMEX   -4.7619  3.230880e+01   
4       10028  2019-10-31 00:00:00     AMEX    6.6667  3.446272e+01   
...       ...                  ...      ...       ...           ...   
79500   93436  2021-11-30 00:00:00   NASDAQ    2.7612  1.149642e+06   
79501   93436  2021-12-31 00:00:00   NASDAQ   -7.6855  1.092218e+06   
79502   93436  2022-01-31 00:00:00   NASDAQ  -11.3609  9.681319e+05   
79503   93436  2022-02-28 00:00:00   NASDAQ   -7.0768  8.996190e+05   
79504   93436  2022-03-31 00:00:00   NASDAQ      -100  1.113736e+06   

            reference_date     gvkey             datadate  book_value  \
0      2019-06-01 00:00:00   12096.0  2018-12-31 00:00:00       8.413   
1

In [49]:
#Construct factors portfolio

#Calculate weighted returns
stocks2018To2022['date'] = pd.to_datetime(stocks2018To2022['date'])
retSizeValue = stocks2018To2022.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retSizeRobust = stocks2018To2022.groupby(['date', 'size_portfolio', 'robust_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retSizeInvestment = stocks2018To2022.groupby(['date', 'size_portfolio', 'investment_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retValueSize = stocks2018To2022.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retRobustSize = stocks2018To2022.groupby(['date', 'robust_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))
retInvestmentSize = stocks2018To2022.groupby(['date', 'investment_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights = pd.to_numeric(x['market_cap_weight'])))

In [50]:
#create factors dataframe
stocks2018To2022.to_sql("stocks2018To2022", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date
        FROM stocks2018To2022
        '''
factors = pd.read_sql(query, conn)
factors['date'] = pd.to_datetime(factors['date'])

In [51]:
#Add smb portfolios based on size
factors['smb_size'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'smb_size'] = np.average(retSizeValue[dateOfData]["S"]) - np.average(retSizeValue[dateOfData]["B"])

#Add smb portfolios based on OP
factors['smb_robust'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'smb_robust'] = np.average(retSizeRobust[dateOfData]["S"]) - np.average(retSizeRobust[dateOfData]["B"])
    
#Add smb portfolios based on INV
factors['smb_investment'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'smb_investment'] = np.average(retSizeInvestment[dateOfData]["S"]) - np.average(retSizeInvestment[dateOfData]["B"])
    
#Add overall smb
factors['smb'] = ""
for i in factors.index:
    factors.loc[i, 'smb'] = np.average([factors.loc[i, 'smb_size'], factors.loc[i, 'smb_robust'], factors.loc[i, 'smb_investment']])


In [56]:
#Add hml portfolios
factors['hml'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'hml'] = np.average(retValueSize[dateOfData]["H"]) - np.average(retValueSize[dateOfData]["L"])

#Add rmw portfolios
factors['rmw'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'rmw'] = np.average(retRobustSize[dateOfData]["R"]) - np.average(retRobustSize[dateOfData]["W"])

#Add cma portfolios
factors['cma'] = ""
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'cma'] = np.average(retInvestmentSize[dateOfData]["C"]) - np.average(retInvestmentSize[dateOfData]["A"])

#Add mkt (monthly average return) across all stocks
factors['mkt'] = ""
weightedAvgMonthlyRet = stocks2018To2022.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights = pd.to_numeric(x.market_cap_weight)))
for i in factors.index:
    dateOfData = factors.loc[i, 'date']
    factors.loc[i, 'mkt'] = weightedAvgMonthlyRet[dateOfData]

In [57]:
print(factors)

         date   smb_size smb_robust smb_investment        smb        hml  \
0  2019-06-28   0.084227   1.423834       0.768773   0.758945   1.574332   
1  2019-07-31  -2.272574  -2.325839      -2.156388    -2.2516  -0.618246   
2  2019-08-30  -3.345323  -4.027537      -4.518393  -3.963751  -4.994083   
3  2019-09-30   0.943966   2.530939        2.06288   1.845928   5.117804   
4  2019-10-31  -1.117675  -1.108091      -1.278526  -1.168097  -2.428583   
5  2019-11-29   0.594385  -0.403561      -0.150056   0.013589  -3.131973   
6  2019-12-31   1.874425   2.572053       2.062585   2.169688   3.164814   
7  2020-01-31  -2.949025  -5.615626      -4.944196  -4.502949  -7.951527   
8  2020-02-28   -1.66665  -2.939667      -2.268595  -2.291638  -3.826731   
9  2020-03-31 -10.499121 -14.278454     -14.627046 -13.134874 -13.697249   
10 2020-04-30   9.312561  10.286617       9.768157   9.789112   8.542743   
11 2020-05-29   3.460979   2.212442       2.430461   2.701294  -3.883293   
12 2020-06-3

In [58]:
#Filter out last month (ret = -100% due to missing dlret data)
factors['date'] = pd.to_datetime(factors['date'])
factors = factors[factors['date'] < "2022-3-31"]

#Parse relevant variables
factors.to_sql("factors", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT date, mkt, smb, hml, rmw, cma
        FROM factors
        '''
factors = pd.read_sql(query, conn)

In [59]:
#sort by date
factors['date'] = pd.to_datetime(factors['date'])
factors = factors.sort_values(by = ['date'], ascending = True)

#Reindex
factors = factors.reset_index()
factors.drop('index', inplace = True, axis = 1)

print(factors)

         date        mkt        smb        hml       rmw       cma
0  2019-06-28   7.814286   0.758945   1.574332  1.816513  1.080462
1  2019-07-31   1.402826  -2.251600  -0.618246 -0.315620  0.105953
2  2019-08-30  -2.229287  -3.963751  -4.994083  0.442176 -1.831736
3  2019-09-30   1.527305   1.845928   5.117804  1.583511  2.592726
4  2019-10-31   2.356417  -1.168097  -2.428583  0.403931  0.182450
5  2019-11-29   3.996196   0.013589  -3.131973 -1.085777 -2.564280
6  2019-12-31   2.998886   2.169688   3.164814  1.611811  2.150192
7  2020-01-31  -0.178745  -4.502949  -7.951527 -1.795483 -2.998006
8  2020-02-28  -7.621064  -2.291638  -3.826731 -2.210369 -1.887208
9  2020-03-31 -12.606939 -13.134874 -13.697249 -0.435595  0.223489
10 2020-04-30  15.424962   9.789112   8.542743  2.927058 -1.230603
11 2020-05-29   5.250147   2.701294  -3.883293 -0.730638 -1.989788
12 2020-06-30   4.051287   4.134458  -3.091025  1.679185  1.089833
13 2020-07-31   6.183695  -1.010140  -2.949189 -0.388509  0.83

In [60]:
#import corresponding factors from Ken French's Study
file = filePath + "/FamaFrenchFiveFactorsMonthlyCopy.csv"
factorsFF = pd.read_csv(file)
print(factorsFF)
print(factors.columns)

       Date  Mkt_RF   SMB   HML   RMW   CMA    RF
0    196307   -0.39 -0.41 -0.97  0.68 -1.18  0.27
1    196308    5.07 -0.80  1.80  0.36 -0.35  0.25
2    196309   -1.57 -0.52  0.13 -0.71  0.29  0.27
3    196310    2.53 -1.39 -0.10  2.80 -2.01  0.29
4    196311   -0.85 -0.88  1.75 -0.51  2.24  0.27
..      ...     ...   ...   ...   ...   ...   ...
706  202205   -0.34 -0.06  8.41  1.44  3.98  0.03
707  202206   -8.43  1.30 -5.97  1.85 -4.70  0.06
708  202207    9.57  1.88 -4.10  0.68 -6.94  0.08
709  202208   -3.78  1.51  0.31 -4.80  1.31  0.19
710  202209   -9.36 -0.96  0.05 -1.40 -0.82  0.19

[711 rows x 7 columns]
Index(['date', 'mkt', 'smb', 'hml', 'rmw', 'cma'], dtype='object')


In [61]:
#clean FF dataframe

#change column names
factorsFF.to_sql("factorsFF", conn, if_exists = 'replace')
query = '''
        SELECT DISTINCT Date as date, Mkt_RF - RF as mkt_ff, SMB as smb_ff, HML as hml_ff, RMW as rmw_ff, CMA as cma_ff
        FROM factorsFF
        '''

factorsFF = pd.read_sql(query, conn)
print(factorsFF)

       date  mkt_ff  smb_ff  hml_ff  rmw_ff  cma_ff
0    196307   -0.66   -0.41   -0.97    0.68   -1.18
1    196308    4.82   -0.80    1.80    0.36   -0.35
2    196309   -1.84   -0.52    0.13   -0.71    0.29
3    196310    2.24   -1.39   -0.10    2.80   -2.01
4    196311   -1.12   -0.88    1.75   -0.51    2.24
..      ...     ...     ...     ...     ...     ...
706  202205   -0.37   -0.06    8.41    1.44    3.98
707  202206   -8.49    1.30   -5.97    1.85   -4.70
708  202207    9.49    1.88   -4.10    0.68   -6.94
709  202208   -3.97    1.51    0.31   -4.80    1.31
710  202209   -9.55   -0.96    0.05   -1.40   -0.82

[711 rows x 6 columns]


In [62]:
#Convert variables to correct type
factorsFF['mkt_ff'] = pd.to_numeric(factorsFF['mkt_ff'], errors = 'coerce')
factorsFF['smb_ff'] = pd.to_numeric(factorsFF['smb_ff'], errors = 'coerce')
factorsFF['hml_ff'] = pd.to_numeric(factorsFF['hml_ff'], errors = 'coerce')
factorsFF['rmw_ff'] = pd.to_numeric(factorsFF['rmw_ff'], errors = 'coerce')
factorsFF['cma_ff'] = pd.to_numeric(factorsFF['cma_ff'], errors = 'coerce')

In [63]:
#Add month column to factors and factorsFF to prepare for left join
factors['date'] = pd.to_datetime(factors['date'])
factors['month'] = factors['date'].dt.month
factorsFF['month'] = ""
for i in factorsFF.index:
    factorsFF.loc[i, 'month'] = int(str(factorsFF.loc[i, 'date'])[-2] + str(factorsFF.loc[i, 'date'])[-1])

In [64]:
#Add year column to factors and factorsFF to prepare for left join
factors['year'] = factors['date'].dt.year
factorsFF['year'] = ""
for i in factorsFF.index:
    factorsFF.loc[i, 'year'] = int(str(factorsFF.loc[i, 'date'])[0:4])


In [65]:
#left join FF results info factors
factors.to_sql("factors", conn, if_exists = 'replace')
factorsFF.to_sql("factorsFF", conn, if_exists = 'replace')
query = '''
        SELECT factors.date, factors.mkt, factors.smb, factors.hml, factors.rmw, factors.cma,
                factorsFF.mkt_ff, factorsFF.smb_ff, factorsFF.hml_ff, factorsFF.rmw_ff, factorsFF.cma_ff
        FROM factors
        LEFT JOIN factorsFF
        ON factors.month = factorsFF.month
        AND factors.year = factorsFF.year
        '''
factors = pd.read_sql(query, conn)
print(factors)

                   date        mkt        smb        hml       rmw       cma  \
0   2019-06-28 00:00:00   7.814286   0.758945   1.574332  1.816513  1.080462   
1   2019-07-31 00:00:00   1.402826  -2.251600  -0.618246 -0.315620  0.105953   
2   2019-08-30 00:00:00  -2.229287  -3.963751  -4.994083  0.442176 -1.831736   
3   2019-09-30 00:00:00   1.527305   1.845928   5.117804  1.583511  2.592726   
4   2019-10-31 00:00:00   2.356417  -1.168097  -2.428583  0.403931  0.182450   
5   2019-11-29 00:00:00   3.996196   0.013589  -3.131973 -1.085777 -2.564280   
6   2019-12-31 00:00:00   2.998886   2.169688   3.164814  1.611811  2.150192   
7   2020-01-31 00:00:00  -0.178745  -4.502949  -7.951527 -1.795483 -2.998006   
8   2020-02-28 00:00:00  -7.621064  -2.291638  -3.826731 -2.210369 -1.887208   
9   2020-03-31 00:00:00 -12.606939 -13.134874 -13.697249 -0.435595  0.223489   
10  2020-04-30 00:00:00  15.424962   9.789112   8.542743  2.927058 -1.230603   
11  2020-05-29 00:00:00   5.250147   2.7

In [66]:
#Compare our results to FF results
factors.describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.9])

,mkt,smb,hml,rmw,cma,mkt_ff,smb_ff,hml_ff,rmw_ff,cma_ff
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,1.896008,0.221635,0.122729,0.718924,0.135290,1.580606,0.045758,0.069394,0.657273,0.421515
std,5.492287,4.615627,5.321149,2.358956,2.064354,5.409492,3.370821,5.121531,2.670903,2.513900
min,-12.606939,-13.134874,-13.697249,-3.213925,-3.065588,-13.520000,-8.310000,-13.970000,-3.800000,-3.250000
10%,-4.353800,-4.293378,-4.771925,-2.050166,-2.163541,-4.224000,-3.886000,-4.864000,-2.060000,-2.244000
25%,-1.025838,-2.273518,-3.091025,-0.730638,-1.736319,-1.550000,-1.780000,-2.170000,-1.470000,-1.250000
50%,2.356417,-0.494658,-0.969985,0.403931,0.105953,2.450000,0.030000,-0.720000,0.300000,-0.430000
75%,4.362704,2.349364,4.879252,2.264357,1.398982,4.620000,1.970000,3.040000,1.840000,1.740000
90%,7.654290,6.417405,5.814492,4.164210,2.457570,6.730000,4.618000,7.012000,4.788000,3.400000
max,15.424962,9.847484,12.752421,5.940222,6.125015,13.650000,7.120000,12.750000,7.220000,7.710000


In [67]:
#kolmogorov-Smirnov test to see if data has statistically significant differences
#mkt
print(ks_2samp(factors['mkt'], factors['mkt_ff']))
print("\n")
#smb
print(ks_2samp(factors['smb'], factors['smb_ff']))
print("\n")
#hml
print(ks_2samp(factors['hml'], factors['hml_ff']))
print("\n")
#rmw
print(ks_2samp(factors['rmw'], factors['rmw_ff']))
print("\n")
#cma
print(ks_2samp(factors['cma'], factors['cma_ff']))

#first number is max difference
#second value is confidence level of the statistic


KstestResult(statistic=0.15151515151515152, pvalue=0.851010941578351, statistic_location=3.75, statistic_sign=-1)


KstestResult(statistic=0.15151515151515152, pvalue=0.851010941578351, statistic_location=-1.0101398179490768, statistic_sign=1)


KstestResult(statistic=0.21212121212121213, pvalue=0.45349473522704375, statistic_location=-2.100157592092054, statistic_sign=1)


KstestResult(statistic=0.15151515151515152, pvalue=0.851010941578351, statistic_location=0.95, statistic_sign=-1)


KstestResult(statistic=0.15151515151515152, pvalue=0.851010941578351, statistic_location=2.592725699564028, statistic_sign=1)


In [68]:
#compute correlations between variables
factors.corr(method = 'pearson')

C:\Users\aaron\AppData\Local\Temp\ipykernel_27140\803550743.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  factors.corr(method = 'pearson')


,mkt,smb,hml,rmw,cma,mkt_ff,smb_ff,hml_ff,rmw_ff,cma_ff
mkt,1.000000,0.570494,0.215424,0.168472,-0.129524,0.994232,0.365760,0.049069,0.243529,-0.239326
smb,0.570494,1.000000,0.534230,-0.192503,0.149015,0.596249,0.930055,0.377863,-0.235925,0.016537
hml,0.215424,0.534230,1.000000,0.318007,0.583148,0.240220,0.469622,0.912263,0.172356,0.602730
rmw,0.168472,-0.192503,0.318007,1.000000,0.136891,0.147008,-0.305288,0.330323,0.917272,0.283139
cma,-0.129524,0.149015,0.583148,0.136891,1.000000,-0.131834,0.170403,0.568249,-0.048843,0.895713
mkt_ff,0.994232,0.596249,0.240220,0.147008,-0.131834,1.000000,0.400966,0.094691,0.225136,-0.243001
smb_ff,0.365760,0.930055,0.469622,-0.305288,0.170403,0.400966,1.000000,0.377847,-0.354256,0.048870
hml_ff,0.049069,0.377863,0.912263,0.330323,0.568249,0.094691,0.377847,1.000000,0.206436,0.613646
rmw_ff,0.243529,-0.235925,0.172356,0.917272,-0.048843,0.225136,-0.354256,0.206436,1.000000,0.112033
cma_ff,-0.239326,0.016537,0.602730,0.283139,0.895713,-0.243001,0.048870,0.613646,0.112033,1.000000
